In [1]:
import os,sys
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import glob as glob
import numpy as np
import warnings
from scipy.stats import ttest_ind,f_oneway
import lightgbm as lgb

from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold, GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,f1_score
from sklearn.pipeline import Pipeline, FeatureUnion

from plot_functions import plot_count, plot_box, plot_box_category, plot_regplot, barplot_count_multicat, \
barplot_pct_multicat, plot_confusion_matrix

%reload_ext autoreload
%autoreload 2
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

warnings.filterwarnings(action='once')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
data_path = os.path.join(os.getcwd(),"Animal State Prediction - dataset")
os.chdir(data_path)
print(data_path)

E:\Projects\ZS_Hackerrank_Feb2020\Animal State Prediction - dataset


In [25]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
display(train_df.head())
display(test_df.head())

,animal_id_outcome,dob_year,dob_month,age_upon_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_outcome,date_of_birth,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,outcome_type
0,A006100,2007,7,10 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Stray,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2017-12-07 00:00:00,12,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194,10 years,2007-07-09 00:00:00,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,12,2017,2017-12,Thursday,0,1.0,Return to Owner
1,A006100,2007,7,7 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,1,2555,7.0,"(5.0, 7.5]",2014-12-19 10:21:00,12,2014,2014-12,Friday,10,2.0,1 days 06:14:00.000000000,1.259722,7 years,2007-07-09 00:00:00,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,12,2014,2014-12,Saturday,16,2.0,Return to Owner
2,A006100,2007,7,6 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,1,2190,6.0,"(5.0, 7.5]",2014-03-07 14:26:00,3,2014,2014-03,Friday,14,3.0,1 days 02:44:00.000000000,1.113889,6 years,2007-07-09 00:00:00,Neutered Male,2190,6.0,"(5.0, 7.5]",NaN,3,2014,2014-03,Saturday,17,3.0,Return to Owner
3,A047759,2004,4,10 years,Dog,Dachshund,Tricolor,Normal,Owner Surrender,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2014-04-02 15:55:00,4,2014,2014-04,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139,10 years,2004-04-02 00:00:00,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,4,2014,2014-04,Monday,15,1.0,Missing
4,A134067,1997,10,16 years,Dog,Shetland Sheepdog,Brown/White,Injured,Public Assist,Neutered Male,1,5840,16.0,"(15.0, 17.5]",2013-11-16 09:02:00,11,2013,2013-11,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444,16 years,1997-10-16 00:00:00,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,11,2013,2013-11,Saturday,11,1.0,Return to Owner


,animal_id_outcome,dob_year,dob_month,age_upon_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_outcome,date_of_birth,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number
0,A723133,2013,3,3 years,Cat,Domestic Shorthair Mix,Brown Tabby,Nursing,Stray,Intact Female,1,1095,3.000000,"(2.5, 5.0]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,5 days 04:48:00.000000000,5.200000,3 years,27-03-2013 00:00,Spayed Female,1095,3.000000,"(2.5, 5.0]",01-04-2016 16:19,4,2016,2016-04,Friday,16,1
1,A723134,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Brown Tabby,Nursing,Stray,Intact Female,1,28,0.076712,"(-0.025, 2.5]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,34 days 06:11:00.000000000,34.257639,2 months,27-02-2016 00:00,Spayed Female,60,0.164384,"(-0.025, 2.5]",30-04-2016 17:42,4,2016,2016-04,Saturday,17,1
2,A723135,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Brown Tabby,Nursing,Stray,Intact Male,1,28,0.076712,"(-0.025, 2.5]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,32 days 12:29:00.000000000,32.520139,2 months,27-02-2016 00:00,Neutered Male,60,0.164384,"(-0.025, 2.5]",29-04-2016 00:00,4,2016,2016-04,Friday,0,1
3,A723136,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Brown Tabby,Nursing,Stray,Intact Female,1,28,0.076712,"(-0.025, 2.5]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,31 days 02:16:00.000000000,31.094444,1 month,27-02-2016 00:00,Intact Female,30,0.082192,"(-0.025, 2.5]",27-04-2016 13:47,4,2016,2016-04,Wednesday,13,1
4,A723137,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Blue Tabby,Nursing,Stray,Intact Female,1,28,0.076712,"(-0.025, 2.5]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,37 days 00:11:00.000000000,37.007639,2 months,27-02-2016 00:00,Intact Female,60,0.164384,"(-0.025, 2.5]",03-05-2016 11:42,5,2016,2016-05,Tuesday,11,1


In [26]:
train_df = train_df[train_df.duplicated(subset=["animal_id_outcome"],keep='first')==False]
train_df.shape

(42679, 37)

In [27]:
y_train = train_df["outcome_type"]

X_train = train_df.drop("outcome_type",axis=1)
print("X_train.shape",X_train.shape)

X_test = test_df
print("X_test.shape",X_test.shape)
X_train["Test"] = False
X_test["Test"] = True

X_train.shape (42679, 36)
X_test.shape (31869, 36)


In [28]:
X = pd.concat([X_train,X_test],axis=0)
print(X.shape,y_train.shape)

(74548, 37) (42679,)


In [29]:
X['sex'] = np.where(X['sex_upon_outcome'].str.contains('Male'), 'Male', 'Female')
X['Spay/Neuter'] = np.where(X['sex_upon_outcome'].str.contains('Intact'), 'No', 'Yes')

X['breed'] = X['breed'].str.lower().str.replace('mix', '').str.replace('medium hair', 'mediumhair').str.strip()

In [31]:
cfa_breeds = ['Abyssinian', 'American Bobtail', 'American Curl', 'American Shorthair', 'American Wirehair', 
              'Balinese', 'Bengal', 'Birman', 'Bombay', 'British Shorthair', 'Burmese', 'Burmilla', 
              'Chartreux', 'Colorpoint Shorthair', 'Cornish Rex', 'Devon Rex', 'Egyptian Mau', 'European Burmese', 
              'Exotic', 'Havana Brown', 'Japanese Bobtail', 'Korat', 'LaPerm', 'Maine Coon', 'Manx', 
              'Norwegian Forest Cat', 'Ocicat', 'Oriental', 'Persian', 'Ragamuffin', 'Ragdoll', 'Russian Blue', 
              'Scottish Fold', 'Selkirk Rex', 'Siamese', 'Siberian', 'Singapura', 'Somali', 'Tonkinese', 
              'Turkish Angora', 'Turkish Van']

cfa_breeds = [i.lower() for i in cfa_breeds]
cfa_breeds = '|'.join(cfa_breeds)

X = pd.concat([X, 
                     X['breed'].str.split('/', expand=True).rename(columns={0:'breed1', 1:'breed2'})], 
                    axis=1)

X['cfa_breed'] = np.where((X['breed1'].str.contains(cfa_breeds)) | 
                                (X['breed2'].str.contains(cfa_breeds)), 
                                True, False)

X['domestic_breed'] = np.where((X['breed1'].str.contains(cfa_breeds) == False) | 
                                     (X['breed2'].str.contains(cfa_breeds) == False), 
                                     True, False)

coat_patterns = ['tabby', 'tortie', 'tortoiseshell', 'calico', 'smoke', 'torbie',
                 'colorpoint', 'point', 'brindle', 'agouti', 'bicolor', 'tricolor']

coat_reg = '(' + '|'.join(coat_patterns) + ')'

X['color'] = X['color'].str.lower()

X['coat_pattern'] = X['color'].str.extract(coat_reg, expand=False)

X['color'] = X['color'].str.replace(coat_reg, '').str.strip()

X = pd.concat([X, 
                     X['color'].str.split('/', expand=True).rename(columns={0:'color1', 1:'color2'})], axis=1)

X['color1'] = np.where(X['color1'] == '', 'Breed Specific', X['color1'])

X['coat'] = np.where(X['color1'] == 'Breed Specific', X['coat_pattern'], X['color1'])

In [35]:
NUM_COLS = [
     'count','age_upon_intake_(days)', 'intake_number', 'time_in_shelter_days','age_upon_outcome_(days)',
"outcome_month","outcome_year","outcome_hour","outcome_number"
 ]

CAT_COLS = [
    'animal_type','intake_condition', 'intake_type', 'sex_upon_intake','sex_upon_outcome',"outcome_monthyear",\
    "outcome_weekday",'sex','Spay/Neuter','breed1','breed2','coat_pattern','color1','color2', 'coat'
]

numerical_df = X[NUM_COLS]
categorical_df = pd.get_dummies(X[CAT_COLS])
test_col_df = X["Test"]

for col in categorical_df.columns:
    if "]" in str(col) or "[" in str(col):
        new_name = str(col).replace("]",")").replace("[","(")
        categorical_df.rename({str(col):new_name},inplace=True,axis=1)
        print(f"Renamed from {col} ---> {new_name}")
        
all_df = pd.concat([numerical_df,categorical_df,test_col_df],axis=1)
print(f"Number of OHE features added: {all_df.shape[1]-len(NUM_COLS)-len(CAT_COLS)-1}")

Number of OHE features added: 745


In [36]:
X_train = all_df[all_df.Test==False]
X_test = all_df[all_df.Test==True]
print(X_train.shape, X_test.shape, y_train.shape)

(42679, 770) (31869, 770) (42679,)


In [37]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb

### XGBoost Best

In [ ]:
best = {'colsample_bylevel':0.7,
        'colsample_bytree':0.8,
        'gamma':.033,
        'learning_rate':0.05,
        'max_depth':68,
        'min_child_weight':4.0,
        'n_estimators':393,
        'reg_alpha':0.45844,
        'reg_lambda':17.7,
        'subsample':0.65    
}

xgb_best = xgb.XGBClassifier(
            colsample_bylevel=best['colsample_bylevel'],
            colsample_bytree=best['colsample_bytree'],
            gamma=best['gamma'],
            learning_rate=best['learning_rate'],
            max_depth=best['max_depth'],
            min_child_weight=best['min_child_weight'],
            n_estimators=best['n_estimators'],
            random_state=0,
            reg_alpha=best['reg_alpha'], 
            reg_lambda=best['reg_lambda'],
            scale_pos_weight=1, 
            seed=None,
            subsample=best['subsample'],
            verbosity=1
)

xtrain, xval, ytrain, yval = train_test_split(X_train,y_train,test_size=0.25,random_state=0)

le = preprocessing.LabelEncoder()
ytrain = le.fit_transform(ytrain)

xgb_best.fit(xtrain,ytrain)
ypred = xgb_best.predict(xval)
y_test_pred_xgb = xgb_best.predict(X_test)

actual = yval
predicted = ypred
predicted = le.inverse_transform(predicted)
predicted_xgb = le.inverse_transform(y_test_pred_xgb)
results = confusion_matrix(actual, predicted) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',f1_score(actual, predicted, average='micro'))
print('Report : ')
print(classification_report(actual, predicted))
#xgb_best.model_save("../models/xgb_best.pkl")

In [59]:
pred_df = pd.DataFrame(predicted_xgb)
pred_df.columns = ["outcome_type"]
pd.concat([test_df["animal_id_outcome"],pred_df],axis=1).to_csv("XGB_pred.csv",index=None)

### Voting Classifier

In [60]:
clf_list = [('randomforest', RandomForestClassifier()),
            ('xgbooost', xgb.XGBClassifier()),
            ('knn', KNeighborsClassifier())]

In [ ]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(clf_list, voting='hard')

xtrain, xval, ytrain, yval = train_test_split(X_train,y_train,test_size=0.25,random_state=0)

voting_clf.fit(xtrain,ytrain)
ypred = voting_clf.predict(xval)
y_test_pred_vot = xgb_best.predict(X_test)

actual = yval
predicted = ypred

In [66]:
#predicted = le.inverse_transform(predicted)
predicted_vot = le.inverse_transform(y_test_pred_vot)
results = confusion_matrix(actual, predicted) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',f1_score(actual, predicted, average='micro'))
print('Report : ')
print(classification_report(actual, predicted))
#xgb_best.model_save("../models/xgb_best.pkl")

pred_df = pd.DataFrame(predicted_vot)
pred_df.columns = ["outcome_type"]
pd.concat([test_df["animal_id_outcome"],pred_df],axis=1).to_csv("Voting_pred.csv",index=None)

Confusion Matrix :
[[2638    3    0    4    2    1   93    2  135]
 [ 365   15    0   26    1    0   27    0  229]
 [   0    0   20   23    0    0    0    0    1]
 [ 108    3    3  667    1    0   99    0  138]
 [ 348    0    0    2    1    2   21    0  218]
 [ 364    1    0    7    1    1   24    1  201]
 [ 233    2    0   21    2    0 1531    0   80]
 [ 329    3    0    6    1    1   17    0  170]
 [ 662   18    0   46   12    6   84    2 1648]]
Accuracy Score : 0.6111527647610122
Report : 
                 precision    recall  f1-score   support

       Adoption       0.52      0.92      0.67      2878
           Died       0.33      0.02      0.04       663
       Disposal       0.87      0.45      0.60        44
     Euthanasia       0.83      0.65      0.73      1019
        Missing       0.05      0.00      0.00       592
       Relocate       0.09      0.00      0.00       600
Return to Owner       0.81      0.82      0.81      1869
      Rto-Adopt       0.00      0.00      0.0

In [65]:
predicted_vot = le.inverse_transform(y_test_pred_vot)